In [29]:
import warnings
import os
from datetime import datetime
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import rc
import platform

# ──────────────────────────────
# Outputs 폴더 생성 + 날짜/시간별 하위 폴더 생성
output_root = "Outputs"
now_str = datetime.now().strftime("%Y-%m-%d_%H-%M")
output_dir = os.path.join(output_root, now_str)
os.makedirs(output_dir, exist_ok=True)

def save_figure(plt_obj, filename):
    """plt 객체를 날짜/시간별 폴더에 그림으로 저장하는 함수"""
    full_path = os.path.join(output_dir, f"{filename}.jpg")
    plt_obj.savefig(full_path, dpi=300, bbox_inches='tight')
    print(f"✅ 그림 저장 완료: {full_path}")

# 고유 의장단수 계산 함수
def count_unique_names(text):
    """직책(이름) 패턴으로 고유 항목 추출 후 개수 반환"""
    if text == 0:
        return 0
    entries = re.findall(r'([가-힣A-Za-z0-9\s/]+?\([가-힣]{2,4}\))', str(text))
    unique_entries = set(entry.strip() for entry in entries if entry.strip())
    return len(unique_entries)

# 한글 폰트 설정
if platform.system() == 'Windows':
    rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else:
    rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

def create_dataframe_from_excel(file_path, sheet_index=0):
    xls = pd.ExcelFile(file_path)
    sheet_name = xls.sheet_names[sheet_index]
    return pd.read_excel(xls, sheet_name=sheet_name, header=0)

def get_excel_file_paths(data_dir):
    return [
        os.path.join(data_dir, f)
        for f in os.listdir(data_dir)
        if f.lower().endswith('.xlsx')
    ]

def remove_front_end_space(df):
    df['표준기구'] = df['표준기구'].str.strip()
    return df

def filter_non_null_rows(df):
    mask = df['소'].notna() & df['순번'].isna()
    df.loc[mask, '순번'] = 0
    def is_valid_seq(x):
        if isinstance(x, str) and x.strip() == '신규':
            return True
        return pd.notna(pd.to_numeric(x, errors='coerce'))
    valid = df['순번'].apply(is_valid_seq)
    return df[df['소'].notna() & valid]

def count_names(text):
    if text == 0:
        return 0
    names = re.split(r'[,]+|\n+', text)
    return len([n.strip() for n in names if n.strip()])

def count_progress_rec(text): return 1 if text == '제정완료' else 0
def count_progress_dev(text): return 1 if text == '개발중' else 0
def count_progress_propose(text): return 1 if text == '제안중' else 0
def count_progress_plan(text): return 1 if text == '계획중' else 0
def count_progress_stop(text): return 1 if text == '개발중단' else 0

def compute_development_stage(row):
    total = row['표준건수 합계']
    if total == 0:
        return 0.0
    plan, propose = row['표준계획 건수'], row['표준제안 건수']
    dev, rec = row['표준개발 건수'], row['표준완료 건수']
    if (plan + propose)/total >= dev/total and (plan + propose)/total >= rec/total:
        return (plan*0.5 + propose*0.5)/total
    if dev/total >= rec/total:
        return 1 + dev/total
    return 2 + rec/total

def extract_position_and_name(text):
    entries = re.findall(r'([가-힣A-Za-z0-9\s/]+?\([가-힣]{2,4}\))', text)
    unique_entries = sorted(set(entry.strip() for entry in entries if entry.strip()))
    return unique_entries

def main(data_dir):
    paths = get_excel_file_paths(data_dir)
    df_std = create_dataframe_from_excel(paths[0])
    df_std = remove_front_end_space(df_std)
    df_std = filter_non_null_rows(df_std)
    print(f"화일의 데이터 행수 ==> {len(df_std)}")

    for col in ['ETRI 기고자', 'ETRI 에디터(예정포함)', 'ETRI 의장단']:
        df_std[col] = df_std[col].fillna(0)

    idx = df_std.columns.get_loc('ETRI 기고자')
    df_std.insert(idx+1, '기고자수', df_std['ETRI 기고자'].apply(count_names))
    idx = df_std.columns.get_loc('ETRI 에디터(예정포함)')
    df_std.insert(idx+1, '에디터수', df_std['ETRI 에디터(예정포함)'].apply(count_names))
    idx = df_std.columns.get_loc('ETRI 의장단')
    df_std.insert(idx+1, '의장단수', df_std['ETRI 의장단'].apply(count_unique_names))

    idx = df_std.columns.get_loc('표준화 상태')
    df_std.insert(idx+1, '표준완료 건수', df_std['표준화 상태'].apply(count_progress_rec))
    df_std.insert(idx+2, '표준개발 건수', df_std['표준화 상태'].apply(count_progress_dev))
    df_std.insert(idx+3, '표준제안 건수', df_std['표준화 상태'].apply(count_progress_propose))
    df_std.insert(idx+4, '표준계획 건수', df_std['표준화 상태'].apply(count_progress_plan))
    df_std.insert(idx+5, '개발중단 건수', df_std['표준화 상태'].apply(count_progress_stop))

    df_std['전략기술 분야'] = df_std['전략기술 분야'].apply(
        lambda v: re.sub(r'^(\d)\)', r'0\1)', v) if isinstance(v, str) else v
    )
    df_std['(예정) 시작년도  '] = df_std['(예정) 시작년도  '].apply(
        lambda v: f"'{int(v)%100:02d}" if isinstance(v, (int,float)) and float(v).is_integer() else v
    )
    df_std['(예정) 완료연도'] = df_std['(예정) 완료연도'].apply(
        lambda v: f"'{int(v)%100:02d}" if isinstance(v, (int,float)) and float(v).is_integer() else v
    )

    # ============================
    # 개발중단된 프로젝트 수 출력
    stopped_count = df_std['개발중단 건수'].sum()
    print(f"🛑 개발 중단된 프로젝트 수: {stopped_count}")

    # ============================
    # 그룹 정의
    group_definitions = [
        (['소'], ['표준완료 건수','표준개발 건수','표준제안 건수','표준계획 건수'], "연차별현황"),
        (['전략기술 분야','세부중점기술 분야','소','본부(단)'], ['표준완료 건수','표준개발 건수','표준제안 건수','표준계획 건수','에디터수','기고자수','표준특허 개수'], "현황총괄_42대중점세부"),
        (['전략기술 분야','소','본부(단)','표준기구'], ['표준완료 건수','표준개발 건수','표준제안 건수','표준계획 건수','의장단수','에디터수','기고자수','표준특허 개수'], "주요참여_표준기구"),
        (['전략기술 분야','소','본부(단)','표준기구'], ['표준완료 건수','표준개발 건수','의장단수','에디터수','기고자수','표준특허 개수'], "표준화 수행 연구부서"),
        (['전략기술 분야','소','본부(단)','표준기구'], ['표준특허 개수'], "표준특허확보"),
        (['전략기술 분야','사업분류'], ['사업분류'], "관련사업현황"),
        (['전략기술 분야'], ['표준완료 건수','표준개발 건수','표준제안 건수','표준계획 건수','의장단수','에디터수','표준특허 개수'], "부록II_12대전략"),
    ]

    grouped_dfs = []
    for group_cols, agg_cols, name in group_definitions:
        if name == "관련사업현황":
            grouped = (
                df_std.groupby(['전략기술 분야', '사업분류'], as_index=False)
                .size()
                .rename(columns={'size': '사업분류수'})
            )
        else:
            grouped = df_std.groupby(group_cols, as_index=False)[agg_cols].sum()
        grouped_dfs.append((grouped, name))

    # ============================
    # 의장단 요약
    strategic_df = df_std[['전략기술 분야', 'ETRI 의장단']].copy()
    strategic_df['ETRI 의장단'] = strategic_df['ETRI 의장단'].fillna('').astype(str).str.strip()
    strategic_df = strategic_df[(strategic_df['ETRI 의장단'] != '') & (strategic_df['ETRI 의장단'] != '0') & (strategic_df['ETRI 의장단'] != '-')]
    strategic_chairs = strategic_df.groupby('전략기술 분야')['ETRI 의장단'].apply(lambda x: ', '.join(x)).reset_index().rename(columns={'ETRI 의장단': '의장단 목록'})
    strategic_chairs['의장단 고유명단'] = strategic_chairs['의장단 목록'].apply(lambda x: ', '.join(extract_position_and_name(x)))
    strategic_chairs['의장단 고유인원수'] = strategic_chairs['의장단 고유명단'].apply(lambda x: len([n for n in x.split(',') if n.strip()]))

    org_df = df_std[['표준기구', 'ETRI 의장단']].copy()
    org_df['ETRI 의장단'] = org_df['ETRI 의장단'].fillna('').astype(str).str.strip()
    org_df = org_df[(org_df['ETRI 의장단'] != '') & (org_df['ETRI 의장단'] != '0') & (org_df['ETRI 의장단'] != '-')]
    org_chairs = org_df.groupby('표준기구')['ETRI 의장단'].apply(lambda x: ', '.join(x)).reset_index().rename(columns={'ETRI 의장단': '의장단 목록'})
    org_chairs['의장단 고유명단'] = org_chairs['의장단 목록'].apply(lambda x: ', '.join(extract_position_and_name(x)))
    org_chairs['의장단 고유인원수'] = org_chairs['의장단 고유명단'].apply(lambda x: len([n for n in x.split(',') if n.strip()]))

    # ============================
    # grouped15 (표준화 단계)
    grouped15 = df_std.groupby(['전략기술 분야','세부중점기술 분야'], as_index=False)[
        ['표준계획 건수','표준제안 건수','표준개발 건수','표준완료 건수','의장단수','에디터수','기고자수']
    ].sum()
    grouped15['표준건수 합계'] = grouped15[['표준계획 건수','표준제안 건수','표준개발 건수','표준완료 건수']].sum(axis=1)
    grouped15['표준개발 단계'] = grouped15.apply(compute_development_stage, axis=1)
    grouped15['표준화 역량 계산값'] = (
        np.log(1 + grouped15['의장단수']*0.1) +
        np.log(1 + grouped15['에디터수']*0.2) +
        np.log(1 + grouped15['기고자수']*0.7)
    ) / 100
    max_by_field = grouped15[~grouped15['세부중점기술 분야'].str.contains('기타')].groupby('전략기술 분야')['표준화 역량 계산값'].transform('max')
    grouped15['표준화 역량'] = (grouped15['표준화 역량 계산값'] / max_by_field) * 3

    grouped15['norm_dev_stage'] = (grouped15['표준개발 단계'] - grouped15['표준개발 단계'].min()) / (grouped15['표준개발 단계'].max() - grouped15['표준개발 단계'].min())
    grouped15['norm_capability'] = (grouped15['표준화 역량 계산값'] - grouped15['표준화 역량 계산값'].min()) / (grouped15['표준화 역량 계산값'].max() - grouped15['표준화 역량 계산값'].min())
    grouped15['종합활동 점수'] = 0.5 * grouped15['norm_dev_stage'] + 0.5 * grouped15['norm_capability']
    grouped15['종합활동 점수(5점척도)'] = (grouped15['종합활동 점수'] * 5).round(0).astype(int)

    grouped_dfs.append((grouped15, "42대중점_전략"))
    
    # ============================
    # Excel 파일 통합 저장
    output_excel_path = os.path.join(output_dir, "통합_출력파일.xlsx")
    with pd.ExcelWriter(output_excel_path, engine="openpyxl") as writer:
        df_std.to_excel(writer, sheet_name="전체데이터", index=False)
        for df, name in grouped_dfs:
            df.to_excel(writer, sheet_name=name, index=False)
        strategic_chairs.to_excel(writer, sheet_name="전략_의장단", index=False)
        org_chairs.to_excel(writer, sheet_name="표준기구_의장단", index=False)
        grouped15.to_excel(writer, sheet_name="42대중점_전략", index=False)
    

    
    print(f"✅ Excel 통합 파일 저장 완료: {output_excel_path}")

    # ============================
    # 그래프 저장
    plt.figure(figsize=(12,8))
    for tech, data in grouped15.groupby('세부중점기술 분야'):
        plt.scatter(data['표준개발 단계'], data['표준화 역량'], label=tech, alpha=0.7)
    plt.xlabel('표준개발 단계', fontsize=12)
    plt.ylabel('표준화 역량', fontsize=12)
    plt.title('세부중점기술 분야별 표준개발 단계와 표준화 역량', fontsize=14)
    plt.xlim(0, 3.5)
    plt.ylim(0, 3.5)
    plt.legend(title='세부중점기술 분야', bbox_to_anchor=(1.05,1), loc='upper left', fontsize=10)
    plt.grid(True)
    plt.tight_layout()
    save_figure(plt, "graph_output")
    plt.show()

    strategic_fields = grouped15['전략기술 분야'].unique()
    for field in strategic_fields:
        df_field = grouped15[grouped15['전략기술 분야'] == field]
        plt.figure(figsize=(12, 8))
        for tech, data in df_field.groupby('세부중점기술 분야'):
            plt.scatter(data['표준개발 단계'], data['표준화 역량'], label=tech, alpha=0.7)
            for idx, row in df_field.iterrows():
                plt.annotate(
                    row['세부중점기술 분야'],
                    (row['표준개발 단계'], row['표준화 역량']),
                    textcoords="offset points",
                    xytext=(5, 5),
                    ha='left',
                    fontsize=9
                )
        plt.xlabel('표준개발 단계', fontsize=12)
        plt.ylabel('표준화 역량', fontsize=12)
        plt.title(f"[{field}] 세부중점기술 분야별 표준개발 단계와 표준화 역량", fontsize=14)
        plt.xlim(0, 3.5)
        plt.ylim(0, 3.5)
        plt.legend(title='세부중점기술 분야', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
        plt.grid(True)
        plt.tight_layout()
        save_figure(plt, f"graph_output_{field}")
        plt.close()
    print("모든 전략기술 분야별 그림이 개별 파일로 저장되었습니다.")

if __name__ == "__main__":
    data_dir = 'data'
    main(data_dir)


화일의 데이터 행수 ==> 650
🛑 개발 중단된 프로젝트 수: 4


NameError: name 'astype' is not defined